### Uploading usefull libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import json
import tpqoa
api = tpqoa.tpqoa("oanda.cfg")

## Creating class for price action strategy

In [2]:
class breakout_swing():
    ''' Class for analyzing failed swings.

    Attributes
    ==========
    ticker: str
        ticker symbol with which to work with
    tf: str
        granularity of data i.e: D, H1, M5
    reward: int
        reward to risk ration given in number
    spread: int
        spread on the ticker used to calculate costs of trading

    Methods
    =======
    check_strategy:
        iterative backtesting method to create df with entered trades
    check_outcome:
        adjusts the trades: droping ones not closed and duplicated entries
    '''
    def __init__(self, reward, spread):
        self.reward = reward
        self.spread = spread
    
    #Add more information about class
    def __repr__(self):
        return "breakout_swing(Reward = {}/1, spread = {})".format(self.reward,self.spread)
    
    def check_strategy(self):
        ''' Iterative backtester through uploaded data
        '''
        self.df = data.copy(deep=True)
        # Create empty df to save all trades made
        self.trades = pd.DataFrame(columns = ['time','position','entry_price','sl','tp','outcome_pips','total_amount_fixed'])
        self.trades.set_index('time', inplace=True)
        # Creating dummy line in trades, this needs to be dropped
        trade = pd.DataFrame([[self.df.index[1],'short',0,0,0,-1,0]], columns = ['time','position','entry_price','sl','tp','outcome_pips','total_amount_fixed'])
        trade.set_index('time', inplace=True)
        self.trades = self.trades.append(trade)
        
        
        start_time = time.time()
        for bar in range(len(self.df)):
            
            # Check if already open trades do not require to be closed:
            # Check long positions:
            if self.trades.outcome_pips.iloc[-1] == 0 and self.trades.position.iloc[-1] == 'long':
                if self.df.l.iloc[bar] < self.trades.sl.iloc[-1]:
                    self.trades.outcome_pips.iloc[-1] = -(self.trades.entry_price.iloc[-1] - self.trades.sl.iloc[-1])
                    #print("{}: Long Position from {} closed | SL: {}".format(self.df.index[bar].date(), self.trades.index[-1].date(), self.trades.sl.iloc[-1]))
                elif self.df.h.iloc[bar] > self.trades.tp.iloc[-1]:
                    self.trades.outcome_pips.iloc[-1] = self.trades.tp.iloc[-1] - self.trades.entry_price.iloc[-1]
                    #print("{}: Long Position from {} closed | TP: {}".format(self.df.index[bar].date(), self.trades.index[-1].date(), self.trades.tp.iloc[-1]))

            # Check short positions:
            elif self.trades.outcome_pips.iloc[-1] == 0 and self.trades.position.iloc[-1] == 'short':
                if self.df.h.iloc[bar] > self.trades.sl.iloc[-1]:
                    self.trades.outcome_pips.iloc[-1] = self.trades.entry_price.iloc[-1] - self.trades.sl.iloc[-1]
                    #print("{}: Short Position from {} closed | SL: {}".format(self.df.index[bar].date(), self.trades.index[-1].date(), self.trades.sl.iloc[-1]))
                elif self.df.l.iloc[bar] < self.trades.tp.iloc[-1]:
                    self.trades.outcome_pips.iloc[-1] = self.trades.entry_price.iloc[-1] - self.trades.tp.iloc[-1]
                    #print("{}: Short Position from {} closed | TP: {}".format(self.df.index[bar].date(), self.trades.index[-1].date(), self.trades.tp.iloc[-1]))
            
            else:
                # Trading only between 7 and 17
                if self.df.hour.iloc[bar] >= 7 and self.df.hour.iloc[bar] <= 17:
                    # If both indicators are equal -1 then sell the asset
                    if self.df.dhs2.iloc[bar] == -1 and self.df.dls2.iloc[bar] == -1:
                        if self.df.l.iloc[bar] < self.df.lows.iloc[bar-1] and self.df.o.iloc[bar] > self.df.lows.iloc[bar-1]:
                            trade = pd.DataFrame([[self.df.index[bar], #time bar
                                                   'short', #short position
                                                   round(self.df.lows.iloc[bar-1] - self.spread, 6), #entry price
                                                   round(self.df.highs.iloc[bar-1] + self.spread, 6), #stop loss price
                                                   round(self.df.lows.iloc[bar-1] - (self.reward * (self.df.highs.iloc[bar-1] + self.spread - (self.df.lows.iloc[bar-1] - self.spread)) - self.spread), 6), # take profit price
                                                   0,0]], # the rest of columns will be filled once trade is closed
                                         columns = ['time','position','entry_price','sl','tp','outcome_pips','total_amount_fixed'])
                            trade.set_index('time', inplace=True)
                            self.trades = self.trades.append(trade)
                            #print("{}: BS_fixed | Go Short | Price: {} | SL: {} | TP: {}".format(self.df.index[bar].date(), self.trades.entry_price[-1], self.trades.sl[-1], self.trades.tp[-1]))

                    # If both indicators are equal 1 then buy the asset
                    elif self.df.dls2.iloc[bar] == 1 and self.df.dhs2.iloc[bar] == 1:
                        if self.df.h.iloc[bar] > self.df.highs.iloc[bar-1] and self.df.o.iloc[bar] < self.df.highs.iloc[bar-1]:
                            trade = pd.DataFrame([[self.df.index[bar], #time bar
                                                   'long', #long position
                                                   round(self.df.highs.iloc[bar-1] + self.spread, 6), #entry price
                                                   round(self.df.lows.iloc[bar-1] - self.spread, 6), #stop loss price
                                                   round(self.df.highs.iloc[bar-1] + (self.reward * (self.df.highs.iloc[bar-1] + self.spread - (self.df.lows.iloc[bar-1] - self.spread)) + self.spread), 6), # take profit price
                                                   0,0]], # the rest of columns will be filled once trade is closed
                                         columns = ['time','position','entry_price','sl','tp','outcome_pips','total_amount_fixed'])
                            trade.set_index('time', inplace=True)
                            self.trades = self.trades.append(trade)
                            #print("{}: BS_fixed | Go Long  | Price: {} | SL: {} | TP: {}".format(self.df.index[bar].date(), self.trades.entry_price[-1], self.trades.sl[-1], self.trades.tp[-1]))
                    
        #print("It took {} minutes to run".format(round((time.time() - start_time)/60)))
        
        
    def check_outcome(self, amount, risk):
        # Dropping trades which were not closed
        self.trades = self.trades[self.trades['outcome_pips'] !=0]
        # Dropping the dummy line
        self.trades = self.trades.iloc[1:,:]
        # Calculating returns
        if self.trades.outcome_pips.iloc[0] < 0:
            self.trades.total_amount_fixed.iloc[0] = amount - amount * risk
        else:
            self.trades.total_amount_fixed.iloc[0] = amount + amount * risk * self.reward

        #### Defining total outcome
        # Defining total outcome fixed for fixed strategy
        for t in range(1, len(self.trades)):
            if self.trades.outcome_pips.iloc[t] < 0:
                self.trades.total_amount_fixed.iloc[t] = self.trades.total_amount_fixed.iloc[t-1] - self.trades.total_amount_fixed.iloc[t-1] * risk
            else:
                self.trades.total_amount_fixed.iloc[t] = self.trades.total_amount_fixed.iloc[t-1] + self.trades.total_amount_fixed.iloc[t-1] * risk * self.reward

        return self.trades

### Generating statistics, accuracy, combined amount.

In [3]:
def close_pos():
    global trades
    print(75 * "-")
    print("+++ CLOSING FINAL POSITION +++")
    print(75 * "-")
    print("+++ Strategy: BreakoutSwingFixed +++")
    print("Instrument: {} | TimeFrame: {} | RiskToReward: {}/{} | Spread: {}".format(ticker, timeframe, int(risk*100), reward, spread))
    print("First trade {}".format(trades.index[0]))
    print("Last trade {}".format(trades.index[-1]))
    txns_fixed = trades.position.count()
    long_trades = trades.loc[trades.position == 'long','position'].count()
    short_trades = trades.loc[trades.position == 'short','position'].count()
    winning_longs = trades.loc[(trades['position']=='long') & (trades['outcome_pips']>0)].position.count()
    winning_shorts = trades.loc[(trades['position']=='short') & (trades['outcome_pips']>0)].position.count()
    print("Number of trades executed = {}".format(txns_fixed))
    print("Long trades executed = {}  |  {}(%) of all trades".format(long_trades, round(long_trades/txns_fixed*100,2)))
    print("Short trades executed = {}  |  {}(%) of all trades".format(short_trades, round((short_trades)/txns_fixed*100,2)))
    print("Winning trades = {}  |  {}(%) of all trades".format(winning_longs + winning_shorts, round((winning_longs + winning_shorts)/txns_fixed*100,2)))
    print("Winning long trades = {}  |  {}(%) of winning trades  |  {}(%) of long trades".format(winning_longs, round(winning_longs/(winning_longs + winning_shorts)*100,2), round(winning_longs/(long_trades)*100,2)))
    print("Winning short trades = {}  |  {}(%) of winning trades  |  {}(%) of short trades".format(winning_shorts, round(winning_shorts/(winning_longs + winning_shorts)*100,2), round(winning_shorts/(short_trades)*100,2)))
    print(25*"+")
    perf = (trades.total_amount_fixed[-1] - amount) / amount * 100
    print("Total Performance of Strategy = {}(%)".format(round(perf, 2)))
    time_traded = int((trades.index[-1] - trades.index[0])/np.timedelta64(1,'D'))/30
    print("Monthly Projection of Performance = {}(%)".format(round(perf/time_traded, 2)))
    print(75 * "-")
    #Create dataframe with testing outcome
    wynik = pd.DataFrame([[ticker,timeframe,reward,spread,txns_fixed,long_trades,short_trades,
                           winning_longs+winning_shorts,winning_longs,winning_shorts,
                           round((winning_longs+winning_shorts)*100/txns_fixed, 2),round(perf, 2),round(perf/time_traded, 2)]],
                         columns = ['asset','timeframe','RR','spread','trades','long_trades','short_trades',
                                    'win_trades','win_longs','win_shorts','Win(%)','Perf(%)','M_Perf(%)'])
    return wynik

# Import prepared dictionary which contains average spreads for all assets
with open('Data\spreads.json', 'r') as fp:
    spreads = json.load(fp)
spreads

# Prepare dataframe to save all outcomes
strategies = pd.DataFrame(columns = ['asset','timeframe','RR','spread','trades','long_trades','short_trades',
                                    'win_trades','win_longs','win_shorts','Win(%)','Perf(%)','M_Perf(%)'])

In [8]:
# Specify size of trading account and risk % wide taken on each trade
# risk to reward ratio is fixed as 3 to 1
amount = 10000
risk = 0.01
reward = 3
timeframe = "M3" # it can be changed to M3
x = 0

In [9]:
instruments = api.get_instruments()
instruments_list = []
for i in instruments:
    instruments_list.append(i[1])
instruments_list

['AUD_CAD',
 'AUD_CHF',
 'AUD_HKD',
 'AUD_JPY',
 'AUD_NZD',
 'AUD_SGD',
 'AUD_USD',
 'AU200_AUD',
 'BCO_USD',
 'DE10YB_EUR',
 'CAD_CHF',
 'CAD_HKD',
 'CAD_JPY',
 'CAD_SGD',
 'CHF_HKD',
 'CHF_JPY',
 'CHF_ZAR',
 'CN50_USD',
 'XCU_USD',
 'CORN_USD',
 'EUR_AUD',
 'EUR_CAD',
 'EUR_CHF',
 'EUR_CZK',
 'EUR_DKK',
 'EUR_GBP',
 'EUR_HKD',
 'EUR_HUF',
 'EUR_JPY',
 'EUR_NOK',
 'EUR_NZD',
 'EUR_PLN',
 'EUR_SEK',
 'EUR_SGD',
 'EUR_TRY',
 'EUR_USD',
 'EUR_ZAR',
 'EU50_EUR',
 'FR40_EUR',
 'GBP_AUD',
 'GBP_CAD',
 'GBP_CHF',
 'GBP_HKD',
 'GBP_JPY',
 'GBP_NZD',
 'GBP_PLN',
 'GBP_SGD',
 'GBP_USD',
 'GBP_ZAR',
 'DE30_EUR',
 'XAU_USD',
 'XAU_AUD',
 'XAU_CAD',
 'XAU_CHF',
 'XAU_EUR',
 'XAU_GBP',
 'XAU_HKD',
 'XAU_JPY',
 'XAU_NZD',
 'XAU_SGD',
 'XAU_XAG',
 'HKD_JPY',
 'HK33_HKD',
 'IN50_USD',
 'JP225_USD',
 'NZD_CAD',
 'NZD_CHF',
 'NZD_HKD',
 'NZD_JPY',
 'NZD_SGD',
 'NZD_USD',
 'NATGAS_USD',
 'NL25_EUR',
 'XPD_USD',
 'XPT_USD',
 'SGD_CHF',
 'SGD_JPY',
 'XAG_USD',
 'XAG_AUD',
 'XAG_CAD',
 'XAG_CHF',
 'XAG_EUR'

In [10]:
j = len(instruments_list)

In [11]:
start_time = time.time()
for ticker in instruments_list:
    x += 1
    data = pd.read_csv("Data/{}_{}_full.csv".format(ticker, timeframe), parse_dates = ["time"], index_col = "time")
    spread = spreads[ticker]
    BS = breakout_swing(reward, spread)
    BS.check_strategy()
    trades = BS.check_outcome(amount, risk)
    outcome = close_pos()
    strategies = strategies.append(outcome)
    trades.to_csv('Wyniki\BS_Fixed_v2\{}_{}_RR{}_trades.csv'.format(ticker,timeframe,reward),index=True)
    print("Part {}/{} done".format(x,j))
strategies.to_csv('Wyniki\BS_Fixed_v2\performances_{}_RR{}.csv'.format(timeframe,reward), index=False)
print("It took {} minutes to run".format(round(time.time() - start_time)/60,2))


---------------------------------------------------------------------------
+++ CLOSING FINAL POSITION +++
---------------------------------------------------------------------------
+++ Strategy: BreakoutSwingFixed +++
Instrument: AUD_CAD | TimeFrame: M3 | RiskToReward: 1/3 | Spread: 0.00023
First trade 2020-01-08 12:27:00
Last trade 2021-02-26 15:54:00
Number of trades executed = 369
Long trades executed = 190  |  51.49(%) of all trades
Short trades executed = 179  |  48.51(%) of all trades
Winning trades = 77  |  20.87(%) of all trades
Winning long trades = 37  |  48.05(%) of winning trades  |  19.47(%) of long trades
Winning short trades = 40  |  51.95(%) of winning trades  |  22.35(%) of short trades
+++++++++++++++++++++++++
Total Performance of Strategy = -48.25(%)
Monthly Projection of Performance = -3.49(%)
---------------------------------------------------------------------------
Part 1/122 done
---------------------------------------------------------------------------
+++ 

---------------------------------------------------------------------------
+++ CLOSING FINAL POSITION +++
---------------------------------------------------------------------------
+++ Strategy: BreakoutSwingFixed +++
Instrument: DE10YB_EUR | TimeFrame: M3 | RiskToReward: 1/3 | Spread: 0.02386
First trade 2020-01-07 10:00:00
Last trade 2021-02-26 16:00:00
Number of trades executed = 278
Long trades executed = 146  |  52.52(%) of all trades
Short trades executed = 132  |  47.48(%) of all trades
Winning trades = 60  |  21.58(%) of all trades
Winning long trades = 29  |  48.33(%) of winning trades  |  19.86(%) of long trades
Winning short trades = 31  |  51.67(%) of winning trades  |  23.48(%) of short trades
+++++++++++++++++++++++++
Total Performance of Strategy = -34.13(%)
Monthly Projection of Performance = -2.46(%)
---------------------------------------------------------------------------
Part 10/122 done
---------------------------------------------------------------------------


---------------------------------------------------------------------------
+++ CLOSING FINAL POSITION +++
---------------------------------------------------------------------------
+++ Strategy: BreakoutSwingFixed +++
Instrument: XCU_USD | TimeFrame: M3 | RiskToReward: 1/3 | Spread: 0.00175
First trade 2020-01-03 15:03:00
Last trade 2021-02-25 17:36:00
Number of trades executed = 323
Long trades executed = 163  |  50.46(%) of all trades
Short trades executed = 160  |  49.54(%) of all trades
Winning trades = 78  |  24.15(%) of all trades
Winning long trades = 43  |  55.13(%) of winning trades  |  26.38(%) of long trades
Winning short trades = 35  |  44.87(%) of winning trades  |  21.88(%) of short trades
+++++++++++++++++++++++++
Total Performance of Strategy = -14.51(%)
Monthly Projection of Performance = -1.04(%)
---------------------------------------------------------------------------
Part 19/122 done
---------------------------------------------------------------------------
+++

---------------------------------------------------------------------------
+++ CLOSING FINAL POSITION +++
---------------------------------------------------------------------------
+++ Strategy: BreakoutSwingFixed +++
Instrument: EUR_HUF | TimeFrame: M3 | RiskToReward: 1/3 | Spread: 0.38818
First trade 2020-01-03 14:39:00
Last trade 2021-02-26 17:54:00
Number of trades executed = 186
Long trades executed = 92  |  49.46(%) of all trades
Short trades executed = 94  |  50.54(%) of all trades
Winning trades = 31  |  16.67(%) of all trades
Winning long trades = 9  |  29.03(%) of winning trades  |  9.78(%) of long trades
Winning short trades = 22  |  70.97(%) of winning trades  |  23.4(%) of short trades
+++++++++++++++++++++++++
Total Performance of Strategy = -47.35(%)
Monthly Projection of Performance = -3.38(%)
---------------------------------------------------------------------------
Part 28/122 done
---------------------------------------------------------------------------
+++ CLOS

---------------------------------------------------------------------------
+++ CLOSING FINAL POSITION +++
---------------------------------------------------------------------------
+++ Strategy: BreakoutSwingFixed +++
Instrument: EUR_ZAR | TimeFrame: M3 | RiskToReward: 1/3 | Spread: 0.0169
First trade 2020-01-02 14:18:00
Last trade 2021-02-24 14:36:00
Number of trades executed = 278
Long trades executed = 141  |  50.72(%) of all trades
Short trades executed = 137  |  49.28(%) of all trades
Winning trades = 51  |  18.35(%) of all trades
Winning long trades = 27  |  52.94(%) of winning trades  |  19.15(%) of long trades
Winning short trades = 24  |  47.06(%) of winning trades  |  17.52(%) of short trades
+++++++++++++++++++++++++
Total Performance of Strategy = -53.88(%)
Monthly Projection of Performance = -3.86(%)
---------------------------------------------------------------------------
Part 37/122 done
---------------------------------------------------------------------------
+++ 

...

---------------------------------------------------------------------------
+++ CLOSING FINAL POSITION +++
---------------------------------------------------------------------------
+++ Strategy: BreakoutSwingFixed +++
Instrument: XAG_EUR | TimeFrame: M3 | RiskToReward: 1/3 | Spread: 0.02158
First trade 2020-01-03 09:06:00
Last trade 2021-02-01 16:48:00
Number of trades executed = 323
Long trades executed = 159  |  49.23(%) of all trades
Short trades executed = 164  |  50.77(%) of all trades
Winning trades = 74  |  22.91(%) of all trades
Winning long trades = 36  |  48.65(%) of winning trades  |  22.64(%) of long trades
Winning short trades = 38  |  51.35(%) of winning trades  |  23.17(%) of short trades
+++++++++++++++++++++++++
Total Performance of Strategy = -27.03(%)
Monthly Projection of Performance = -2.05(%)
---------------------------------------------------------------------------
Part 82/122 done
---------------------------------------------------------------------------
+++

---------------------------------------------------------------------------
+++ CLOSING FINAL POSITION +++
---------------------------------------------------------------------------
+++ Strategy: BreakoutSwingFixed +++
Instrument: TRY_JPY | TimeFrame: M3 | RiskToReward: 1/3 | Spread: 0.03546
First trade 2020-01-07 08:48:00
Last trade 2021-02-26 09:48:00
Number of trades executed = 156
Long trades executed = 61  |  39.1(%) of all trades
Short trades executed = 95  |  60.9(%) of all trades
Winning trades = 37  |  23.72(%) of all trades
Winning long trades = 7  |  18.92(%) of winning trades  |  11.48(%) of long trades
Winning short trades = 30  |  81.08(%) of winning trades  |  31.58(%) of short trades
+++++++++++++++++++++++++
Total Performance of Strategy = -9.73(%)
Monthly Projection of Performance = -0.7(%)
---------------------------------------------------------------------------
Part 91/122 done
---------------------------------------------------------------------------
+++ CLOSIN

---------------------------------------------------------------------------
+++ CLOSING FINAL POSITION +++
---------------------------------------------------------------------------
+++ Strategy: BreakoutSwingFixed +++
Instrument: SPX500_USD | TimeFrame: M3 | RiskToReward: 1/3 | Spread: 0.38066
First trade 2020-01-06 14:30:00
Last trade 2021-02-26 14:48:00
Number of trades executed = 420
Long trades executed = 233  |  55.48(%) of all trades
Short trades executed = 187  |  44.52(%) of all trades
Winning trades = 98  |  23.33(%) of all trades
Winning long trades = 56  |  57.14(%) of winning trades  |  24.03(%) of long trades
Winning short trades = 42  |  42.86(%) of winning trades  |  22.46(%) of short trades
+++++++++++++++++++++++++
Total Performance of Strategy = -28.78(%)
Monthly Projection of Performance = -2.07(%)
---------------------------------------------------------------------------
Part 100/122 done
---------------------------------------------------------------------------

---------------------------------------------------------------------------
+++ CLOSING FINAL POSITION +++
---------------------------------------------------------------------------
+++ Strategy: BreakoutSwingFixed +++
Instrument: USD_HUF | TimeFrame: M3 | RiskToReward: 1/3 | Spread: 0.37365
First trade 2020-01-03 13:12:00
Last trade 2021-01-07 12:24:00
Number of trades executed = 171
Long trades executed = 88  |  51.46(%) of all trades
Short trades executed = 83  |  48.54(%) of all trades
Winning trades = 35  |  20.47(%) of all trades
Winning long trades = 13  |  37.14(%) of winning trades  |  14.77(%) of long trades
Winning short trades = 22  |  62.86(%) of winning trades  |  26.51(%) of short trades
+++++++++++++++++++++++++
Total Performance of Strategy = -28.27(%)
Monthly Projection of Performance = -2.3(%)
---------------------------------------------------------------------------
Part 109/122 done
---------------------------------------------------------------------------
+++ C

---------------------------------------------------------------------------
+++ CLOSING FINAL POSITION +++
---------------------------------------------------------------------------
+++ Strategy: BreakoutSwingFixed +++
Instrument: USD_TRY | TimeFrame: M3 | RiskToReward: 1/3 | Spread: 0.01468
First trade 2020-01-03 10:45:00
Last trade 2021-02-26 14:51:00
Number of trades executed = 85
Long trades executed = 50  |  58.82(%) of all trades
Short trades executed = 35  |  41.18(%) of all trades
Winning trades = 18  |  21.18(%) of all trades
Winning long trades = 11  |  61.11(%) of winning trades  |  22.0(%) of long trades
Winning short trades = 7  |  38.89(%) of winning trades  |  20.0(%) of short trades
+++++++++++++++++++++++++
Total Performance of Strategy = -13.18(%)
Monthly Projection of Performance = -0.94(%)
---------------------------------------------------------------------------
Part 118/122 done
---------------------------------------------------------------------------
+++ CLOS

In [14]:
strategies = pd.read_csv('Wyniki\BS_Fixed_v2\performances_{}_RR{}.csv'.format(timeframe,reward))
strategies

,asset,timeframe,RR,spread,trades,long_trades,short_trades,win_trades,win_longs,win_shorts,Win(%),Perf(%),M_Perf(%)
0,AUD_CAD,M5,3,0.00023,254,133,121,42,22,20,16.54,-58.90,-4.29
1,AUD_CHF,M5,3,0.00027,210,108,102,41,24,17,19.52,-38.53,-2.75
2,AUD_HKD,M5,3,0.00143,286,156,130,67,32,35,23.43,-19.79,-1.42
3,AUD_JPY,M5,3,0.01743,234,136,98,61,39,22,26.07,6.65,0.48
4,AUD_NZD,M5,3,0.00029,221,111,110,51,24,27,23.08,-18.21,-1.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,USD_TRY,M3,3,0.01468,85,50,35,18,11,7,21.18,-13.18,-0.94
240,USD_ZAR,M3,3,0.01221,300,157,143,59,25,34,19.67,-49.25,-3.54
241,WTICO_USD,M3,3,0.03004,353,195,158,86,42,44,24.36,-13.18,-0.94
242,WHEAT_USD,M3,3,0.01065,170,83,87,38,19,19,22.35,-18.41,-1.35


In [15]:
strategies_m3 = strategies[strategies['timeframe'] == "M3"]
strategies_m5 = strategies[strategies['timeframe'] == "M5"]

In [22]:
best_m3 = strategies_m3.sort_values(by = ['Perf(%)'], ascending=False).head(10)
best_m3

,asset,timeframe,RR,spread,trades,long_trades,short_trades,win_trades,win_longs,win_shorts,Win(%),Perf(%),M_Perf(%)
160,FR40_EUR,M3,3,1.02440,358,164,194,104,45,59,29.05,68.43,4.89
169,GBP_USD,M3,3,0.00017,337,172,165,97,51,46,28.78,57.64,4.12
232,USD_JPY,M3,3,0.01427,376,176,200,106,45,61,28.19,52.14,3.72
184,HK33_HKD,M3,3,5.03802,361,189,172,101,51,50,27.98,45.11,3.25
171,DE30_EUR,M3,3,2.25221,434,236,198,117,61,56,26.96,31.31,2.24
164,GBP_HKD,M3,3,0.00195,324,156,168,89,47,42,27.47,30.85,2.21
146,EUR_DKK,M3,3,0.00118,111,52,59,34,0,34,30.63,26.00,1.93
211,SUGAR_USD,M3,3,0.00027,140,72,68,41,18,23,29.29,24.23,1.82
223,US30_USD,M3,3,1.87774,298,160,138,81,39,42,27.18,23.78,2.30
141,CORN_USD,M3,3,0.01012,90,45,45,28,15,13,31.11,22.69,1.71


In [19]:
best_m5 = strategies_m5.sort_values(by = ['Perf(%)'], ascending=False).head(10)
best_m5

,asset,timeframe,RR,spread,trades,long_trades,short_trades,win_trades,win_longs,win_shorts,Win(%),Perf(%),M_Perf(%)
99,SPX500_USD,M5,3,0.38066,224,119,105,73,39,34,32.59,89.68,6.45
38,FR40_EUR,M5,3,1.02440,193,96,97,60,26,34,31.09,54.78,3.95
110,USD_JPY,M5,3,0.01427,249,126,123,73,35,38,29.32,47.54,3.44
97,NAS100_USD,M5,3,1.47737,273,168,105,77,50,27,28.21,35.82,2.58
98,US2000_USD,M5,3,0.37792,275,143,132,77,43,34,28.00,33.12,2.39
101,US30_USD,M5,3,1.87774,233,137,96,66,36,30,28.33,31.32,3.08
24,EUR_DKK,M5,3,0.00118,83,39,44,27,0,27,32.53,26.52,1.94
9,DE10YB_EUR,M5,3,0.02386,192,104,88,54,28,26,28.12,23.27,1.68
119,WTICO_USD,M5,3,0.03004,208,107,101,58,32,26,27.88,22.98,1.65
58,XAU_NZD,M5,3,1.10764,212,108,104,59,30,29,27.83,22.91,1.65
